In [2]:
import mikeio as mi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gdp
from shapely.geometry import Point, LineString
import sys
from math import atan2, degrees, hypot
import shapely.wkt
import shapely.ops

def reverse_geom(geom):
    def _reverse(x, y, z=None):
        if z:
            return x[::-1], y[::-1], z[::-1]
        return x[::-1], y[::-1]

    return shapely.ops.transform(_reverse, geom)

def get_grid_size(ds):
    # Check if quadratic grid
    if ds.geometry.dx != ds.geometry.dy:
        print('The grid is not quadratic. This might cause insufficient results')
    return ds.geometry.dx


def AngleBtw2Points(pointA, pointB):
    changeInX = pointB[0] - pointA[0]
    changeInY = pointB[1] - pointA[1]
    return degrees(atan2(changeInY, changeInX))

def get_values_for_linestring(grid_size, ds, geometry_row):

    # Number of points in the polyline
    num_coords = len(geometry_row.coords)

    # Convert gridsize to integer
    grid_size = int(grid_size)

    # Loop through each coordinate, create a new polyline for each and extract result
    for j in range(1, num_coords):

        # Define start and end coordinates
        starting_coord = geometry_row.coords[j-1]
        end_coord = geometry_row.coords[j]

        # If this is the first coordinate in the linestring, create a new result_df that will later be concatenated with the other ones.
        val_dict = {}
        if j == 1:
            # Extract results
            point_array = ds.sel(x=starting_coord[0], y=starting_coord[1])
            time = point_array.time
            items = point_array.items
            val_dict['time'] = time

            # Extract results for each item in the dfs2
            for item in items:
                val_1 = point_array[item].values
                val_dict[str(item)] = val_1
            df_result = pd.DataFrame(index=time, data=val_dict)
        # Round the length of the line to closest integer and convert
        length_of_line = int(
            np.round(Point(starting_coord).distance(Point(end_coord)), 0))

        # Create new LineString of new coordinates
        sub_line = LineString([starting_coord, end_coord])
        df_dict = {}

        # Loop thorugh the values in the dfs2 file, with the step size equivalent to the gridsize
        for i in range(0, length_of_line, grid_size):

            # Reset the values dictionary
            val_dict = {}

            # Get coordinates of point with i distance from start coordinate
            ip = sub_line.interpolate(distance=i)

            # Extract results from point
            point_array = ds.sel(x=ip.coords[0][0], y=ip.coords[0][1])
            time = point_array.time
            items = point_array.items
            val_dict['time'] = time

            # Extract results for each item in the dfs2
            for item in items:
                val_1 = point_array[item].values
                val_dict[str(item)] = val_1

            # Concatenate the results of the new coordinate to result_df. Note that here they are concatenated vertically
            # The vertical concatenation will be later dissolved using the groupby argument (for the time column) and taken as mean
            df_next_coord = pd.DataFrame(index=time, data=val_dict)
            df_result = pd.concat([df_result, df_next_coord])

    # df_result = df_result.groupby('time', as_index=False).mean()
    # For swedish users, the , is the default separator, instead of . (This is a feature requested by the user)
    columns = df_result.columns
    # for col in columns:
    #    df_result[col] = df_result[col].astype(
    #        str).str.replace('.', ',', regex=False)

    return df_result


def dfs2tocsv(filepath_ds, filepath_shape):

    # Check whether it is a dfs2 file or not
    if filepath_ds[-4:] != 'dfs2':
        print("Please pick a dfs2 file")
        return False

    # Read dfs2
    ds = mi.read(filepath_ds)

    # Check whether it is a shp file or not
    if filepath_shape[-3:] != 'shp':
        print("Please pick a shapefile")
        return False

    # Read shapefile
    shape_df = gdp.read_file(filepath_shape)

    # Check whether the file is a Polyline
    if not isinstance(shape_df['geometry'][0], LineString):
        print('The file is not a Polyline, please specify a PolyLine instead.')
        return False

    # Get grid size
    grid_size = get_grid_size(ds)

    # Dictionary with results
    df_dict = {}

    # Loop through df with the shapefile and save the results
    for index, row in shape_df.iterrows():

        # Check direction. The calculations should always be done from left to right.
        first_coord = row['geometry'].coords[0]
        last_coord = row['geometry'].coords[len(row['geometry'].coords)-1]
        angle = AngleBtw2Points(first_coord,last_coord)

        #If angle is in the second or third quadrant, the general direction is right, so it must be reversed 
        if (angle>90) | (angle<-90):
            row['geometry'] = reverse_geom(row['geometry'])
            print('The general directions is right')
        df_dict[index] = get_values_for_linestring(
            grid_size, ds, row['geometry'])

    # Create a multilevel indexed dataframe for easier read in the csv format (requested by users)
    resultant_df = pd.concat(df_dict.values(), axis=1, keys=df_dict.keys())
    # resultant_df.to_csv(output_dir_and_name, sep=';')
    return resultant_df


# if __name__ == '__main__':
#    dfs2tocsv(sys.argv[1], sys.argv[2], sys.argv[3])


In [3]:
filepath_ds = r'C:\Project folder\python_specialuppdrag\MASV specialuppdrag\06_Linnestaden_HPQ.dfs2'
filepath_shp = r'C:\Project folder\python_specialuppdrag\MASV specialuppdrag\endast_en.shp'
#test = dfs2tocsv(filepath_ds,filepath_shp)
ds = mi.read(filepath_ds)
shape_df = gdp.read_file(filepath_shp)
grid_size= get_grid_size(ds)
test = get_values_for_linestring(grid_size, ds, shape_df['geometry'][0])
#test_df = dfs2tocsv(filepath_ds,filepath_shp)

In [4]:
test

,time,Total water depth <Water Depth> (meter),P flux <Flow Flux> (meter pow 3 per sec per meter),Q flux <Flow Flux> (meter pow 3 per sec per meter)
2014-07-26 17:45:00,2014-07-26 17:45:00,0.000000,0.000000e+00,0.000000e+00
2014-07-26 17:50:00,2014-07-26 17:50:00,0.001721,0.000000e+00,0.000000e+00
2014-07-26 17:55:00,2014-07-26 17:55:00,0.002221,0.000000e+00,0.000000e+00
2014-07-26 18:00:00,2014-07-26 18:00:00,0.002575,0.000000e+00,0.000000e+00
2014-07-26 18:05:00,2014-07-26 18:05:00,0.002181,7.050004e-11,-4.344904e-15
...,...,...,...,...
2014-07-27 05:40:00,2014-07-27 05:40:00,0.000562,0.000000e+00,0.000000e+00
2014-07-27 05:45:00,2014-07-27 05:45:00,0.000562,0.000000e+00,0.000000e+00
2014-07-27 05:50:00,2014-07-27 05:50:00,0.000562,0.000000e+00,0.000000e+00
2014-07-27 05:55:00,2014-07-27 05:55:00,0.000562,0.000000e+00,0.000000e+00


In [67]:
from math import atan2,degrees
line = shape_df['geometry'][0]
lineList = list(line.coords)

def AngleBtw2Points(pointA, pointB):
  changeInX = pointB[0] - pointA[0]
  changeInY = pointB[1] - pointA[1]
  return degrees(atan2(changeInY,changeInX)) #remove degrees if you want your answer in radians

angle_line = AngleBtw2Points(lineList[0],lineList[1])
perpendicular_line = angle_line+90

In [30]:
x = test.iloc[900,2]
y = test.iloc[900,3]
test.iloc[900]

time                                                  2014-07-26 18:45:00
Total water depth <Water Depth> (meter)                           2.16943
P flux <Flow Flux> (meter pow 3 per sec per meter)              -0.096181
Q flux <Flow Flux> (meter pow 3 per sec per meter)               0.239225
Name: 2014-07-26 18:45:00, dtype: object

In [31]:
from math import hypot

In [32]:
hypot(x,y)
degrees(atan2(x,y))

0.2578364273676064

In [12]:
test['Resultant_flow'] = test.apply(lambda x: hypot(x['P flux <Flow Flux> (meter pow 3 per sec per meter)'],x['Q flux <Flow Flux> (meter pow 3 per sec per meter)']),axis=1)
test['degrees'] = test.apply(lambda x:degrees(atan2(x['P flux <Flow Flux> (meter pow 3 per sec per meter)'],x['Q flux <Flow Flux> (meter pow 3 per sec per meter)'])),axis=1)

In [13]:
test

,time,Total water depth <Water Depth> (meter),P flux <Flow Flux> (meter pow 3 per sec per meter),Q flux <Flow Flux> (meter pow 3 per sec per meter),hypot,degrees,Resultant_flow
2014-07-26 17:45:00,2014-07-26 17:45:00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
2014-07-26 17:50:00,2014-07-26 17:50:00,0.001721,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
2014-07-26 17:55:00,2014-07-26 17:55:00,0.002221,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
2014-07-26 18:00:00,2014-07-26 18:00:00,0.002575,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
2014-07-26 18:05:00,2014-07-26 18:05:00,0.002181,7.050004e-11,-4.344904e-15,7.050004e-11,90.003531,7.050004e-11
...,...,...,...,...,...,...,...
2014-07-27 05:40:00,2014-07-27 05:40:00,0.000562,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
2014-07-27 05:45:00,2014-07-27 05:45:00,0.000562,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
2014-07-27 05:50:00,2014-07-27 05:50:00,0.000562,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
2014-07-27 05:55:00,2014-07-27 05:55:00,0.000562,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00


#### NOTE TO AFTERNOON! if angle == (perpendicular+- 180): positive ,else negative (sort of) 


- No we have to extract at which SIDE the extraction point is from and then done. 

In [55]:
test2=ds.sel(x = lineList[0][0],y=lineList[0][1])

In [59]:
test2.geometry

GeometryPoint2D(x=147275.75, y=6395986.750004)

In [60]:
test2.geometry.x

147275.75

In [61]:
extracted_point_x = test2.geometry.x
extracted_point_y = test2.geometry.y

In [62]:
extracted_point_y

6395986.750004

In [64]:
atan2(x,y)

-0.38227428775020417

In [68]:
angle_line

-158.7026459548892

In [79]:
import shapely.wkt
import shapely.ops


def reverse_geom(geom):
    def _reverse(x, y, z=None):
        if z:
            return x[::-1], y[::-1], z[::-1]
        return x[::-1], y[::-1]

    return shapely.ops.transform(_reverse, geom)

test3 = reverse_geom(line)

In [82]:
test3.wkt

'LINESTRING (147230.33088362962 6395969.038813498, 147277.16222729348 6395987.295100007)'

In [83]:
line.wkt

'LINESTRING (147277.16222729348 6395987.295100007, 147230.33088362962 6395969.038813498)'